In [19]:
import pandas as pd
import numpy as np
df = pd.read_csv("hospitaldata.csv", sep=",", na_values="-")

In [20]:
# Q1.  remove the dots in the names
df = df.rename(columns=lambda x: x.replace('.', ' '))
df = df.rename(columns=lambda x: x.replace('  ', '_'))
df = df.rename(columns=lambda x: x.replace(' ', '_'))
print df.columns

Index([u'Date', u'id', u'Time', u'Age', u'Sex', u'Consulting_Doctor',
       u'Specialty', u'Procedure', u'Total_Charges', u'Amount_Received_',
       u'Amount_Balance', u'Amount_Received_By', u'Amount_in_Hospital',
       u'Receptionist_Name', u'Next_Apt'],
      dtype='object')


In [21]:
# Q2. Which day of the week is expected to have most visits? Ans == Monday 

new_df = pd.DataFrame(df.Date.str.split(',',1).tolist(), columns = ['Day','Date'])
print new_df['Day'].value_counts().max()
print new_df['Day'].value_counts().idxmax()

51
Monday


In [22]:
# 3.  What is the average age of patients? Ans== 32.7
df['Age'] = pd.to_numeric(df.Age, errors='coerce')
print df['Age'].mean()

32.734375


In [23]:
# 4. How many children were entertained? (Make a Bracket of Age from 1-12) ? Ans == 23
print len(list(filter(lambda x: x < 12, df['Age'])))

23


In [24]:
# 5. Which speciality has most patients? Ans == Dentist

df['Sex'] = map(lambda x: str(x).replace("-", ""), df['Sex'])
df['Sex'] = map(lambda x: str(x).upper(), df['Sex'])
print df.groupby(['Sex']).Specialty.value_counts()

Sex  Specialty         
F    Dentist               57
     Gynae                 13
     M/o                    7
     Orthopedic             5
     Psychotherapist        4
     Child Specialist       3
     Radiologist            3
     General Surgeon        2
     Dental                 1
     ENT                    1
     Medical Specialist     1
M    Dentist               46
     Psychotherapist       10
     M/o                    8
     Child Specialist       7
     Orthopedic             3
     Radiologist            3
     General Surgeon        2
     Medical Specialist     2
     Anaesthetist           1
     Dermatologist          1
NAN  Child Specialist       1
Name: Specialty, dtype: int64


In [25]:
# 6. Which Doctor is earning highest? Alaf khan, 513050

df['Total_Charges'] = pd.to_numeric(df['Total_Charges'], errors='coerce')
print df.groupby(['Consulting_Doctor']).Total_Charges.sum().max()
print df.groupby(['Consulting_Doctor']).Total_Charges.sum().idxmax()


513050.0
Dr Alaf Khan


In [26]:
# 7. Which procedure type earns more money? Ans== Orthodontics , 240000

print df.groupby(['Procedure']).Total_Charges.sum().max()
print df.groupby(['Procedure']).Total_Charges.sum().idxmax()

240000.0
Orthodontics


In [27]:
# 8. Which time of the day has highest frequency of visits by hour ? Ans == 13 , 29
new_time = df[['Time']].copy()
new_time.Time[0] = new_time.Time[0]+"AM"

df.Time = new_time.Time
df.Time = df.Time.replace('-',np.nan)
df.Time = pd.to_datetime(df.Time,format = '%I:%M%p')
# creating hour colur
df['Hour'] = df.Time.dt.hour

visits = df[['Hour']].copy()
visits['Visits']=0
visits = visits.groupby(['Hour']).agg({'Visits':"count"})
visits = visits.sort_values(by=['Visits'],ascending=False)
print "The time with highets frequency: ";
visits.head(n=1)

The time with highets frequency: 


,Visits
Hour,
13.0,29


In [28]:
#10. How many patients are repeated visitors? Ans = 37

print sum(df['id'].value_counts() > 1)

37


In [29]:
# 11.  Give us the id of repeated visitors. 
## ANS ==  [1, 4, 12, 13, 17, 20, 25, 40, 45, 46, 59, 63, 64, 80, 88, 94, 96, 97, 100, 101, 107, 109, 112, 114, 116, 118, 120, 122, 130, 132, 133, 140, 145, 149, 150, 151, 153]
from collections import Counter
freq = Counter(df['id'])
# print freq
ids = []
for key in freq:
    if (freq[key]) > 1:
        ids.append(key)
print ids

[1, 4, 12, 13, 17, 20, 25, 40, 45, 46, 59, 63, 64, 80, 88, 94, 96, 97, 100, 101, 107, 109, 112, 114, 116, 118, 120, 122, 130, 132, 133, 140, 145, 149, 150, 151, 153]


In [30]:
# 12. Which patients visited again for the same problem?

dd = df.groupby(['id']).Specialty.value_counts()[df.groupby(['id']).Specialty.value_counts() > 1]
print dd

id   Specialty      
12   Dentist            2
13   Orthopedic         2
17   Dentist            2
25   Gynae              2
40   Dentist            2
45   Dentist            3
63   Dentist            2
88   Dentist            2
96   Dentist            2
97   Psychotherapist    2
100  Dentist            2
101  Gynae              3
107  Dentist            2
109  Dentist            3
112  Orthopedic         2
114  Gynae              3
116  Dentist            2
120  M/o                2
130  Psychotherapist    2
132  Dentist            3
140  Dentist            4
145  Dentist            3
149  Dentist            2
151  Dentist            2
153  Dentist            2
Name: Specialty, dtype: int64


In [31]:
# 13. What is the median age for Females and Males? Ans == M=29 , F = 30
print df.groupby(['Sex']).Age.median()

Sex
F      30.0
M      29.0
NAN     NaN
Name: Age, dtype: float64


In [32]:
# 14. What is the total amount in balance? Ans == 222500
df['Amount_Balance'] = map(lambda x: str(x).replace(",", ""), df['Amount_Balance'])
df['Amount_Balance'] = pd.to_numeric(df['Amount_Balance'], errors='coerce')
print df.Amount_Balance.sum()

222500.0


In [33]:
# 15. How much money was made by Procedure Type “Consultation”?  Ans == 83950
print df.groupby('Procedure')['Total_Charges'].sum()["Consultation"]

83950.0


In [34]:
# 16.   Is there a relation between Age and Total Charges paid?
# Ans = No, Cor == 0.028
print df['Age'].corr(df['Total_Charges'])

0.028090455626


In [35]:
# Q 17. Which Age group had highest number of visits?
# Ans == 30
import operator
freq_age = Counter(df['Age'])
print max(freq_age.iteritems(), key=operator.itemgetter(1))

(30.0, 20)


In [36]:
# Q 18. What is the total cost earned by Procedure Type X Ray and Scalling together?
#Ans == 22300
print df.groupby('Procedure')['Total_Charges'].sum()["X Ray"] + df.groupby('Procedure')['Total_Charges'].sum()["Scalling"]

22300.0


In [38]:
# Printing clean data set

df.to_csv("clean_hosp_data.csv", sep=',')

